In [1]:
'''Script to extract and sort data of mini ruedi measurments from .txt files for STANDARDS'''

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import plotly.express as px
import os
from datetime import datetime


from pathlib import Path

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
#sample group directory that should contain all standards
directory = "/Users/eart0477/Documents/mac_jupyter_files/yellowstone2022/Yellowstone22/raw_data/all_standards/"

In [3]:
#sort files into categories from a folder. 

txt_files = []
for file in os.listdir(directory):
    if file.endswith(".txt"):
        txt_files.append(os.path.join(file))
        
sample_list = []
standard_list = []
blank_list = []

for string in txt_files:
    if "SAMPLE" in string:
        sample_list.append(string)
    elif "STANDARD" in string:
        standard_list.append(string)
    elif "BLANK" in string:
        blank_list.append(string)


In [4]:
def processing_function(file_path):   
    
    mz_values = set()

    with open(file_path) as f:



    #extract all mz values        
        for line in f:
            if 'mz=' in line and 'concentration=' not in line:
                mz_start = line.index('mz=') + 3
                space_index = line.index(' ', mz_start)
                mz = line[mz_start:space_index]
                mz_values.add(mz)

    mz_array = list(mz_values)


    #Scan through txt to extract detector and intesity values of every line and store in a nested list grouped by mz

    combined_lists = {mz: {'F': {'PEAK': [], 'ZERO': []}, 'M': {'PEAK': [], 'ZERO': []}} for mz in mz_array}

    with open(file_path) as f:
        for line in f:
            if 'mz=' in line and 'concentration=' not in line:
                mz_start = line.index('mz=') + 3
                space_index = line.index(' ', mz_start)
                mz = line[mz_start:space_index]

                detector_start = line.index('detector=') + 9
                space_index = line.index(' ', detector_start)
                detector = line[detector_start:space_index]

                intensity_start = line.index('intensity=') + 10
                space_index = line.index(' ', intensity_start)
                intensity = line[intensity_start:space_index]

                reading = 'PEAK' if 'PEAK' in line else 'ZERO'

                combined_lists[mz][detector][reading].append(intensity)

    #make a dataframe 

    df = pd.DataFrame(columns=['mz', 'detector', 'reading', 'intensity'])

    for mz in combined_lists:
        for detector in combined_lists[mz]:
            for reading in combined_lists[mz][detector]:
                for intensity in combined_lists[mz][detector][reading]:
                    df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)


    #make sure intensity is numeric                
    df['intensity'] = pd.to_numeric(df['intensity'])

    
     #find sample_name  
    with open(file_path) as f:
        
        # Extract the base filename and extension from the file path
        filename, ext = os.path.splitext(os.path.basename(file_path))

        # Remove spaces from the filename
        file_name = filename.replace(' ', '')
        # find sample_name
        for line in f:
            if 'SAMPLENAME:' in line:
                name_start = line.index('SAMPLENAME:') + 12
                space_index = line.index(' ', name_start)
                sample_name = line[name_start:space_index].strip()
                break
            else:
                sample_name = None 

        # if 'SAMPLENAME:' not found, scan the file again to find 'ANALYSISTYPE:'
        if sample_name is None:
                sample_name = file_name
               
            

       


        return sample_name, df




In [5]:
def find_mean(sample_name, df):     
    #find mean and std of measurments

        # group the data frame by the 'mz', 'detector' columns
        grouped = df.groupby(['mz', 'detector', 'reading'])

        # calculate the mean and standard deviation of the 'intensity' column for each group
        result = grouped['intensity'].agg(['mean', 'std'])

        # reset the index of the result data frame to make the 'mz' and 'detector' columns columns again
        result = result.reset_index()
        
        peak_F = result[(result['detector'] == 'F') & (result['reading'] == 'PEAK')]



        #Selecting background on M detector
        peak_M = result[(result['detector'] == 'M') & (result['reading'] == 'PEAK')]
        
        final = pd.concat([peak_F, peak_M])
        


        # Initialize an empty dictionary to store the reformatted data
        data_dict = {}

        # Loop through the rows of the original dataframe
        for i, row in final.iterrows():
            # Extract the mz value
            mz = row['mz']

            # Construct column names for mean and std
            mean_col = f"{mz}_F_mean"
            std_col = f"{mz}_F_std"

            # Extract mean and std values
            mean_val = row['mean']
            std_val = row['std']

            # Add the mean and std values to the data_dict
            data_dict[mean_col] = mean_val
            data_dict[std_col] = std_val

        # Create a new dataframe from the data_dict
        new_df = pd.DataFrame([data_dict], index=[sample_name])

      
        
        return final

In [6]:
# Define a list of column names
cols = ['date', '4_F_mean', '4_F_std', '4_M_mean', '4_M_std', '28_F_mean', '28_F_std', '32_F_mean', '32_F_std',
        '36_M_mean', '36_M_std','40_F_mean', '40_F_std', '40_M_mean', '40_M_std',
        '44_F_mean', '44_F_std',  '84_M_mean', '84_M_std', '14_F_mean', '14_F_std', '15_F_mean', '15_F_std', '16_F_mean', '16_F_std',
        '18_F_mean', '18_F_std',]

# Create an empty dataframe with the specified columns
full_df = pd.DataFrame(columns=cols)

# Iterate through a list of standards
standard_count = 0        
for i in standard_list: 
    standard_count += 1
    path = directory + i
        
    # Run all processing functions for each sample
    sample_name, df = processing_function(path)
    result = find_mean(sample_name, df)
        
    # Initialize an empty dictionary to store the reformatted data
    data_dict = {}
    
    

    # Loop through the rows of the original dataframe
    for i, row in result.iterrows():
        # Extract the mz value
        mz = row['mz']
        detector = row['detector']

        # Construct column names for mean and std
        mean_col = f"{mz}_{detector}_mean"
        std_col = f"{mz}_{detector}_std"

        # Extract mean and std values
        mean_val = row['mean']
        std_val = row['std']

        # Add the mean and std values to the data_dict
        data_dict[mean_col] = mean_val
        data_dict[std_col] = std_val
        
        
    

    # Create a new dataframe from the data_dict
    new_df = pd.DataFrame([data_dict], index=[sample_name])

    # Append the new row to the full_df dataframe
    full_df = full_df.append(new_df)




/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)
/var/folde

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/1463440100.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = full_df.append(new_df)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: Future

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/1463440100.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = full_df.append(new_df)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: Future

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/1463440100.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = full_df.append(new_df)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: Future

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/1463440100.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = full_df.append(new_df)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: Future

...

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/1463440100.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = full_df.append(new_df)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: Future

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/1463440100.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = full_df.append(new_df)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: Future

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/1463440100.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = full_df.append(new_df)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: Future

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/1463440100.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = full_df.append(new_df)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: Future

/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/1463440100.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = full_df.append(new_df)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'mz': mz, 'detector': detector, 'reading': reading, 'intensity': intensity}, ignore_index=True)
/var/folders/4g/gxf8tc9x34jfnsd6g2p5z3ym0000gq/T/ipykernel_21687/2267112829.py:51: Future

In [ ]:
for i, index in full_df.iterrows():
    dt_obj = datetime.strptime(i, '%Y-%m-%d_%H-%M-%S_STANDARD')
    full_df.loc[i, 'date'] = dt_obj
    full_df.loc[i, 'day'] = dt_obj.day

full_df.sort_values(by='date', inplace=True)
# Save the dataframe to an Excel file
#filename = 'all_yellowstone_standards.xlsx'
#full_df.to_excel(filename)

In [11]:
# Define the columns to group by and calculate the mean and standard deviation of
group_cols = ['day']
mean_cols = [col for col in full_df.columns if 'mean' in col]

# Group the dataframe by day and calculate the mean of the mean_cols
grouped_df = full_df.groupby(group_cols)[mean_cols].mean()

# Calculate the standard deviation of the mean_cols for each day
std_cols = [col.replace('mean', 'std') for col in mean_cols]
std_values = full_df.groupby(group_cols)[std_cols].std()
for col in std_cols:
    mean_col = col.replace('std', 'mean')
    # Fill missing values with NaN
    mean_values = grouped_df[mean_col].fillna(np.nan)
    # Calculate the percentage of standard deviation with respect to the mean
for col in std_cols:
    mean_col = col.replace('std', 'mean')
    if col in std_values:
        grouped_df[col] =  std_values[col].divide(grouped_df[mean_col], fill_value=np.nan)

In [156]:

# Save the dataframe
#filename = 'yellowstone_standard_reproducility.xlsx'
#grouped_df.to_excel(filename)

In [12]:

fig = px.scatter()
fig.add_scatter(x= full_df['date'], y=full_df['28_F_mean'], mode='markers', error_y=dict(array=full_df['28_F_std']), name='28_F_mean')
fig.update_layout(xaxis_title="Date")
fig.update_layout(title_text='mz 28 F', title_x=0.5)
fig.update_yaxes(tickformat=".2e")


In [13]:

fig = px.scatter()
fig.add_scatter(x= full_df['date'], y=full_df['15_F_mean'], mode='markers', error_y=dict(array=full_df['15_F_std']), name='15_F_mean')
fig.update_layout(xaxis_title="Date")
fig.update_layout(title_text='mz 15 F', title_x=0.5)
fig.update_yaxes(tickformat=".2e")

In [14]:
fig = px.scatter()
fig.add_scatter(x= full_df['date'], y=full_df['44_F_mean'], mode='markers', error_y=dict(array=full_df['44_F_std']), name='44_F_mean')
fig.update_layout(xaxis_title="Date")
fig.update_layout(title_text='mz 44 F', title_x=0.5)
fig.update_yaxes(tickformat=".2e")

In [15]:
fig = px.scatter()
fig.add_scatter(x= full_df['date'], y=full_df['4_F_mean'], mode='markers', error_y=dict(array=full_df['4_F_std']), name='4_F_mean')
fig.update_layout(xaxis_title="Date")
fig.update_layout(title_text='mz 4 F', title_x=0.5)
fig.update_yaxes(tickformat=".2e")

In [16]:
fig = px.scatter()
fig.add_scatter(x= full_df['date'], y=full_df['4_M_mean'], mode='markers', error_y=dict(array=full_df['4_M_std']), name='4_M_mean')
fig.update_layout(xaxis_title="Date")
fig.update_layout(title_text='mz 4 M', title_x=0.5)
fig.update_yaxes(tickformat=".2e")

In [17]:
fig = px.scatter()
fig.add_scatter(x= full_df['date'], y=full_df['20_M_mean'], mode='markers', error_y=dict(array=full_df['20_M_std']), name='20_M_mean')
fig.update_layout(xaxis_title="Date")
fig.update_layout(title_text='mz 20 M', title_x=0.5)
fig.update_yaxes(tickformat=".2e")

In [18]:
fig = px.scatter()
fig.add_scatter(x= full_df['date'], y=full_df['36_M_mean'], mode='markers', error_y=dict(array=full_df['36_M_std']), name='36_M_mean')
fig.update_layout(xaxis_title="Date")
fig.update_layout(title_text='mz 36 M', title_x=0.5)
fig.update_yaxes(tickformat=".2e")

In [19]:
fig = px.scatter()
fig.add_scatter(x= full_df['date'], y=full_df['40_M_mean'], mode='markers', error_y=dict(array=full_df['40_M_std']), name='40_M_mean')
fig.update_layout(xaxis_title="Date")
fig.update_layout(title_text='mz 40 M', title_x=0.5)
fig.update_yaxes(tickformat=".2e")

In [20]:
fig = px.scatter()
fig.add_scatter(x= full_df['date'], y=full_df['40_F_mean'], mode='markers', error_y=dict(array=full_df['40_F_std']), name='40_F_mean')
fig.update_layout(xaxis_title="Date")
fig.update_layout(title_text='mz 40 F', title_x=0.5)
fig.update_yaxes(tickformat=".2e")

In [21]:
fig = px.scatter()
fig.add_scatter(x= full_df['date'], y=full_df['32_F_mean'], mode='markers', error_y=dict(array=full_df['32_F_std']), name='32_F_mean')
fig.update_layout(xaxis_title="Date")
fig.update_layout(title_text='mz 32 F', title_x=0.5)
fig.update_yaxes(tickformat=".2e")

In [22]:
fig = px.scatter()
fig.add_scatter(x= full_df['date'], y=full_df['84_M_mean'], mode='markers', error_y=dict(array=full_df['84_M_std']), name='84_M_mean')
fig.update_layout(xaxis_title="Date")
fig.update_layout(title_text='mz 84 M', title_x=0.5)
fig.update_yaxes(tickformat=".2e")

,date,4_F_mean,4_F_std,4_M_mean,4_M_std,28_F_mean,28_F_std,32_F_mean,32_F_std,36_M_mean,36_M_std,40_F_mean,40_F_std,40_M_mean,40_M_std,44_F_mean,44_F_std,84_M_mean,84_M_std,14_F_mean,14_F_std,15_F_mean,15_F_std,16_F_mean,16_F_std,18_F_mean,18_F_std,20_M_mean,20_M_std
2022-09-09_05-06-52_STANDARD,NaN,1.510000e-14,7.351871e-15,1.646386e-11,4.928016e-13,9.742739e-10,7.059671e-12,1.958475e-10,9.059885e-13,2.008040e-10,2.710570e-12,1.602858e-11,1.003955e-13,7.007129e-08,7.206545e-10,1.037100e-12,1.780225e-14,3.511290e-12,2.482343e-13,4.337136e-11,2.859839e-13,1.574400e-13,5.315355e-15,1.424000e-11,7.932062e-14,1.027636e-11,3.581547e-14,4.604150e-09,4.862609e-11
2022-09-13_05-40-46_STANDARD,NaN,6.920000e-15,5.806204e-15,1.474995e-11,3.875000e-13,1.071243e-09,5.157471e-12,2.254442e-10,1.772306e-12,2.099169e-10,3.876301e-12,1.817312e-11,1.219219e-13,7.800981e-08,5.157198e-10,1.600080e-12,3.052420e-14,2.973600e-12,9.909456e-14,4.376684e-11,2.390913e-13,1.282800e-13,5.372802e-15,1.412602e-11,1.090740e-13,1.238800e-11,1.311951e-13,4.625991e-09,7.147139e-11
2022-09-18_05-20-46_STANDARD,NaN,1.300000e-14,6.052272e-15,2.095367e-11,2.414626e-13,1.317158e-09,7.976755e-12,2.811565e-10,1.629333e-12,2.911331e-10,2.675786e-12,2.303082e-11,9.660935e-14,1.043833e-07,1.758302e-09,1.856160e-12,9.902929e-15,6.497710e-12,1.590097e-13,5.347226e-11,1.432510e-13,1.629400e-13,7.559630e-15,1.724296e-11,1.361444e-13,2.776304e-11,1.720650e-13,6.399184e-09,1.200062e-10
2022-09-18_02-18-59_STANDARD,NaN,1.980000e-15,3.892557e-15,2.239102e-11,3.834218e-13,1.453961e-09,7.221543e-12,3.076222e-10,1.824458e-12,3.027308e-10,2.642552e-12,2.385668e-11,1.604123e-13,1.068697e-07,2.920553e-10,2.170000e-12,9.527067e-15,6.910180e-12,1.692564e-13,5.738662e-11,3.794884e-13,1.721600e-13,6.913610e-15,1.855478e-11,1.322696e-13,1.991372e-11,9.451707e-14,6.776307e-09,6.359166e-11
2022-09-21_03-31-35_STANDARD,NaN,4.920000e-15,5.291692e-15,1.361111e-11,4.150510e-13,1.171277e-09,9.484408e-12,2.322002e-10,1.255792e-12,2.257372e-10,2.858517e-12,1.880962e-11,8.931600e-14,8.428426e-08,5.132110e-10,1.953500e-12,2.099202e-14,2.706710e-12,1.075156e-13,4.155736e-11,2.561593e-13,1.145200e-13,1.029063e-14,1.326488e-11,7.804548e-14,2.130006e-11,3.302067e-13,4.927141e-09,4.129115e-11
2022-09-16_04-53-26_STANDARD,NaN,1.320000e-14,4.009364e-15,2.092094e-11,3.482430e-13,1.422418e-09,4.380205e-12,3.025600e-10,3.262746e-12,2.943386e-10,3.789588e-12,2.474102e-11,1.404383e-13,1.046973e-07,1.419948e-09,2.186620e-12,2.135280e-14,5.829820e-12,2.118557e-13,5.644410e-11,2.458202e-13,1.714000e-13,8.677269e-15,1.854324e-11,7.039725e-14,1.687468e-11,4.110172e-13,6.494348e-09,7.418781e-11
2022-09-13_02-57-35_STANDARD,NaN,9.800000e-16,6.302142e-15,1.180043e-11,1.575777e-13,1.157356e-09,6.860983e-12,2.336426e-10,9.209915e-13,1.668517e-10,1.618922e-12,1.826190e-11,7.940765e-14,6.379909e-08,1.039741e-09,1.671720e-12,2.724403e-14,3.633140e-12,1.689709e-13,4.165948e-11,1.216546e-13,1.187800e-13,4.688497e-15,1.413770e-11,1.039037e-13,1.800102e-11,3.811895e-13,3.680635e-09,1.971737e-11
2022-09-15_04-41-56_STANDARD,NaN,1.994000e-14,4.826282e-15,1.883810e-11,7.114414e-13,1.437765e-09,1.423270e-11,2.977938e-10,1.593324e-12,2.719218e-10,4.800422e-12,2.348290e-11,1.155237e-13,9.581749e-08,6.407141e-10,2.535900e-12,5.603619e-14,5.277490e-12,1.356463e-13,5.461332e-11,3.270635e-13,1.771000e-13,8.355537e-15,1.773060e-11,1.307507e-13,2.933486e-11,1.150754e-12,6.212775e-09,5.522188e-11
2022-09-21_01-44-06_STANDARD,NaN,-5.420000e-15,3.423010e-15,1.481498e-11,2.488628e-13,1.246222e-09,9.103701e-12,2.483358e-10,2.104896e-12,2.398078e-10,3.521466e-12,1.958638e-11,3.652242e-14,8.802212e-08,8.478834e-10,2.121420e-12,3.419221e-14,3.466210e-12,1.311175e-13,4.446006e-11,2.958891e-13,1.243200e-13,4.769382e-15,1.431690e-11,6.495002e-14,2.306424e-11,1.288619e-12,5.338308e-09,7.696144e-11
2022-09-21_04-21-53_STANDARD,NaN,1.312000e-14,2.248777e-15,1.464540e-11,4.448167e-13,1.159469e-09,6.029728e-12,2.318213e-10,1.418669e-12,2.299845e-10,3.049633